## PAGERANK 

As we have seen there is a clear positive skew in the business of airports across the country. We have seen that **ORD** or **ATL** are two of the busiest airports, while there are hundreds of other airports that operate on a much smaller scale.

##### Goal:
Use the NetworkX Graph library to analyze the underlying flow of traffic at all US airports via Pagerank.

##### Hypothesis:
By calculating the **Pagerank**(popularity) of an airport, we are able to capture the network flow that exists amongst airports in the US. This score would reflect the random chance of traveller ending up at a given airport. We hope to see a meaningful correlation between Pagerank and `DEP_DEL15` whether that is positive or negative. As we get to modelling stages, we will experiment with `ORIGIN_PAGERANK` and `DEST_PAGERANK` both as features.

In [0]:
import networkx as nx
import numpy as np
import pandas as pd
import plotly.express as px
import collections
import math


import pyspark.sql.functions as f
from pyspark.sql.types import *
from pyspark.sql.functions import isnan, when, count, col

### Pull in cleaned flights data

In [0]:
airlines = spark.read.option("header", "true").parquet(f"dbfs:/mnt/mids-w261/team20SSDK/cleaned_data/airlines/airlines_latest_utc/part-00*.parquet")

In [0]:
#Because between query may have edge case, I'm just being explicit here
air_w_year = airlines.select("ORIGIN", "ORIGIN_UTC", "DEST", "DEST_UTC",  
                              f.year("ORIGIN_UTC").alias('utc_org_year'), 
                              f.year("DEST_UTC").alias('utc_dest_year'),                              
                             )
print(air_w_year.count())
display(air_w_year)

31171199

ORIGIN,ORIGIN_UTC,DEST,DEST_UTC,utc_org_year,utc_dest_year
BNA,2019-07-31T15:45:00.000+0000,MSP,2019-07-31T17:55:00.000+0000,2019,2019
BNA,2019-07-31T23:15:00.000+0000,MSP,2019-08-01T01:20:00.000+0000,2019,2019
BNA,2019-08-01T00:45:00.000+0000,MSY,2019-08-01T02:05:00.000+0000,2019,2019
BNA,2019-07-31T18:30:00.000+0000,MSY,2019-07-31T20:05:00.000+0000,2019,2019
BNA,2019-07-31T12:45:00.000+0000,MSY,2019-07-31T14:15:00.000+0000,2019,2019
BNA,2019-07-31T21:45:00.000+0000,MSY,2019-07-31T23:15:00.000+0000,2019,2019
BNA,2019-07-31T12:50:00.000+0000,OAK,2019-07-31T17:25:00.000+0000,2019,2019
BNA,2019-08-01T00:10:00.000+0000,PHL,2019-08-01T02:00:00.000+0000,2019,2019
BNA,2019-07-31T15:25:00.000+0000,PHL,2019-07-31T17:30:00.000+0000,2019,2019
BNA,2019-08-01T03:05:00.000+0000,PHL,2019-08-01T05:00:00.000+0000,2019,2019


### Build this metric only based off training data

In [0]:
airlines = air_w_year.where(f.col("utc_org_year") < 2019)
airlines.select("utc_org_year").distinct().collect()

Out[13]: [Row(utc_org_year=2015),
 Row(utc_org_year=2016),
 Row(utc_org_year=2018),
 Row(utc_org_year=2017),
 Row(utc_org_year=2014)]

### Convert the sample to Pandas
* To use NetworkX
* Ease of visual building

In [0]:

airlines_pd = airlines.toPandas()
airlines_pd.head()

Out[14]:

,ORIGIN,ORIGIN_UTC,DEST,DEST_UTC,utc_org_year,utc_dest_year
0,PHL,2017-08-24 11:05:00,MCO,2017-08-24 13:34:00,2017,2017
1,PHL,2017-08-25 11:05:00,MCO,2017-08-25 13:34:00,2017,2017
2,PHL,2017-08-26 11:05:00,MCO,2017-08-26 13:34:00,2017,2017
3,PHL,2017-08-27 11:05:00,MCO,2017-08-27 13:34:00,2017,2017
4,PHL,2017-08-28 11:05:00,MCO,2017-08-28 13:34:00,2017,2017


In [0]:
airlines_pd.shape

Out[15]: (23900414, 6)

### Get all unique airports in out flights data

In [0]:
total_unique_airports = list(set(airlines_pd["ORIGIN"]).union(set(airlines_pd["DEST"])))
len(total_unique_airports)

Out[16]: 365

### Initialize NetworkX Directed Graph, and add all the airports as nodes

In [0]:
#Init A Directed Graph
G = nx.DiGraph()

#Add all airports as nodes
G.add_nodes_from(total_unique_airports)

### Group all `ORIGIN -> DEST` flight paths and get the counts

We will use these counts as the weights on this directed graph. Where the weight of an edge from node A to Node B is the number of times a flight has made this trip in the time range 2015-2018.

In [0]:
grouped_by_trip = airlines_pd.groupby(['ORIGIN','DEST']).size().reset_index()
grouped_by_trip = grouped_by_trip.rename(columns={0: "count"})
grouped_by_trip.head()

Out[18]:

,ORIGIN,DEST,count
0,ABE,ATL,3790
1,ABE,CLT,911
2,ABE,DTW,3201
3,ABE,EWR,1
4,ABE,FLL,72


#### Add all the edges of graph with weight as the total number of times a given trip has occurred.

In [0]:
#For every origin and destination add edge with weight which represents total number of trips across all time 2015-2019
for row in grouped_by_trip.iterrows():
  row_obj = row[1]
  G.add_edge(row_obj["ORIGIN"], row_obj["DEST"], weight = row_obj["count"])

#### Compute the Page Rank of the airports

In [0]:
%%time

airport_pr = nx.pagerank(G, alpha=0.85, personalization=None, max_iter=1000, tol=1e-06, nstart=None, weight='weight', dangling=None)
airport_pr_sorted = {k: v for k, v in sorted(airport_pr.items(), key=lambda item: -item[1])}

CPU times: user 158 ms, sys: 65.8 ms, total: 223 ms
Wall time: 224 ms

In [0]:
airport_pr_sorted

Out[13]: {'ATL': 0.05675633205934823,
 'ORD': 0.04567374440718648,
 'DFW': 0.03817081837439715,
 'DEN': 0.036128662676027073,
 'LAX': 0.028122056024440168,
 'SFO': 0.023987106854337407,
 'MSP': 0.023149781459302094,
 'IAH': 0.022271477655767678,
 'PHX': 0.02217095769474956,
 'DTW': 0.021234275775648754,
 'CLT': 0.020704596327078387,
 'SEA': 0.020256383440287556,
 'LAS': 0.01975077705019088,
 'SLC': 0.01932465232346387,
 'MCO': 0.017148024939902334,
 'BOS': 0.015792086073036823,
 'EWR': 0.015434598617683535,
 'LGA': 0.014765007085961535,
 'JFK': 0.014062128214538082,
 'BWI': 0.012545744129215625,
 'FLL': 0.01201245396638502,
 'PHL': 0.01151870330781668,
 'DCA': 0.011473750629657485,
 'MDW': 0.010834396753342789,
 'SAN': 0.010450935249211379,
 'MIA': 0.009831010390197664,
 'TPA': 0.009156497013769234,
 'DAL': 0.008640774471484973,
 'PDX': 0.007919740160974442,
 'HNL': 0.007805844403420148,
 'BNA': 0.00747256690184683,
 'STL': 0.007389252100360184,
 'HOU': 0.007176568374778409,
 'ANC': 0.007129840416576604,
 'AUS': 0.0068127326626474015,
 'IAD': 0.006401502954303568,
 'OAK': 0.006356369329348691,
 'MCI': 0.0061529917648698446,
 'MSY': 0.006126440774752534,
 'SJC': 0.006044960474743502,
 'SMF': 0.005695904332237595,
 'SNA': 0.005538200598155632,
 'RDU': 0.0054485990900004005,
 'CLE': 0.00513207743191577,
 'SAT': 0.004589133849041476,
 'IND': 0.004552593378627481,
 'MKE': 0.004419197653450909,
 'PIT': 0.004391938220902758,
 'RSW': 0.004186061831878509,
 'CMH': 0.003976180990117637,
 'CVG': 0.003889779910182605,
 'SJU': 0.003577886385520175,
 'OGG': 0.0035629037678561247,
 'PBI': 0.0034059059581220547,
 'BDL': 0.0031753699808742763,
 'BUR': 0.003157212681933998,
 'JAX': 0.0030723278612960403,
 'OMA': 0.0030075981177967,
 'ABQ': 0.0029346078766914834,
 'JNU': 0.0029310332148858873,
 'ONT': 0.0028562468817290338,
 'BUF': 0.002749348350526399,
 'OKC': 0.0026939328084901524,
 'RIC': 0.002633162222625401,
 'SFB': 0.0026006907453628315,
 'MEM': 0.0025683778372551517,
 'TUS': 0.0024980563772376277,
 'BOI': 0.002451366996262799,
 'TUL': 0.0023751025391687843,
 'CHS': 0.0023382245923860723,
 'RNO': 0.0022897550539230545,
 'GRR': 0.002155315884540878,
 'KOA': 0.0021275029838059905,
 'SDF': 0.0021225820735515997,
 'BHM': 0.0021112911129121076,
 'ORF': 0.002104063109129402,
 'PVD': 0.002096136115420124,
 'LGB': 0.002047328901582513,
 'LIH': 0.002031358772664868,
 'ELP': 0.001999797430208354,
 'GEG': 0.0018568771858964643,
 'DSM': 0.0018288404816986575,
 'TYS': 0.0017612933476494348,
 'MSN': 0.0017395444054666015,
 'LIT': 0.0017142995962787168,
 'MYR': 0.001702855455301311,
 'AZA': 0.0016821899204729704,
 'PSP': 0.0016792946982938665,
 'SAV': 0.0016400964493916842,
 'PIE': 0.0016313116323640742,
 'ALB': 0.001592510490842694,
 'ICT': 0.001591364507488358,
 'DAY': 0.0015880334788090755,
 'COS': 0.001575630142063662,
 'ROC': 0.0015582581872749623,
 'XNA': 0.0015319386889370205,
 'GSP': 0.0015278943580705373,
 'FAT': 0.001523084092354844,
 'KTN': 0.0014704559044644216,
 'SYR': 0.0014401486011411805,
 'GSO': 0.0014182320081870187,
 'CID': 0.0014070111008404177,
 'LEX': 0.0013936414733277628,
 'HPN': 0.0013680384961274777,
 'BTR': 0.0013542309621350842,
 'PNS': 0.0013360343025440643,
 'FAI': 0.0013236156237751,
 'ITO': 0.0012740992192146665,
 'PWM': 0.0012664697932481318,
 'MAF': 0.0012581771038164275,
 'FWA': 0.0012558613287652733,
 'SBN': 0.0012531380235737287,
 'MHT': 0.0012415218354880375,
 'JAN': 0.0012258072478493786,
 'PGD': 0.0012211242042744449,
 'VPS': 0.0012144228110371794,
 'SHV': 0.0012107168570046044,
 'FSD': 0.0012064185069901748,
 'SGF': 0.0011961273715167106,
 'CAK': 0.0011948020441749847,
 'SBA': 0.0011547532409955858,
 'CHA': 0.0011467379891484354,
 'FAR': 0.0011359895256935644,
 'HSV': 0.0010981575270876138,
 'BZN': 0.0010924916367343709,
 'AVL': 0.001089043855907684,
 'MOB': 0.0010772621367289583,
 'SIT': 0.0010677984038649018,
 'CAE': 0.001063014059327267,
 'PSG': 0.0010615841742818402,
 'FNT': 0.0010539702432268527,
 'PIA': 0.00

#### Some graph metrics

In [0]:
#2015-2018
print("Number of total edges: ", G.number_of_edges())

print("Number of total trips : ", G.size(weight='weight'))

print("Sanity check with grouped trips", sum(grouped_by_trip["count"]))

Number of total edges: 6665
Number of total trips : 23900414.0
Sanity check with grouped trips 23900414

### Aiport Edge Degree

We wanted to gain an understanding of the behavior as per the **In-Degree** and **Out-Degree** for all airports we are analyzing

#### In-Degree Analysis

In [0]:
in_degree_sequence = sorted([d for n, d in G.in_degree], reverse=True)  # degree sequence
in_degree_count = collections.Counter(in_degree_sequence)
in_deg, in_cnt = zip(*in_degree_count.items())

in_degree_count = pd.DataFrame({"in_degree": in_deg, "count":in_cnt})

in_degree_count.describe()

Out[42]:

,in_degree,count
count,79.000000,79.000000
mean,52.860759,4.620253
std,43.089571,9.641396
min,0.000000,1.000000
25%,19.500000,1.000000
50%,44.000000,1.000000
75%,75.500000,3.000000
max,185.000000,60.000000


In [0]:
fig = px.bar(in_degree_count, x='in_degree', y='count',
             hover_data=['in_degree', 'count'], color='count', height=400,
             title="Airport In-Degree Count")

fig.show()

#### Out-Degree Analysis

In [0]:
out_degree_sequence = sorted([d for n, d in G.out_degree], reverse=True)  # degree sequence
out_degree_count = collections.Counter(out_degree_sequence)
out_degree, out_cnt = zip(*out_degree_count.items())
out_degree_count = pd.DataFrame({"out_degree": out_degree, "count":out_cnt})

out_degree_count.describe()

Out[44]:

,out_degree,count
count,74.000000,74.000000
mean,53.229730,4.932432
std,43.777315,9.829131
min,0.000000,1.000000
25%,18.250000,1.000000
50%,45.000000,2.000000
75%,76.250000,4.000000
max,184.000000,59.000000


In [0]:
fig = px.bar(out_degree_count, x='out_degree', y='count',
             hover_data=['out_degree', 'count'], color='count', height=400,
             title="Airport Out-Degree Count")

fig.show()

### We have computed a Page rank for every airport

Pull in `airport_meta` table and add this as a new data point for every airport that we have seen in our training data.

In [0]:
airport_meta = spark.read.option("header", "true").parquet(f"dbfs:/mnt/mids-w261/team20SSDK/cleaned_data/station/airport_meta/part-00000*.parquet")
print(airport_meta.count())
display(airport_meta)

369

ICAO,IATA,usaf,wban,name,country,state,lat,lon,elev,begin,end,STATION,station_tz,pagerank
KBGM,BGM,725150,04725,GREATER BINGHAMTON/E A LINK F,US,NY,42.207,-75.98,+0486.2,19730101,20190305,72515004725,America/New_York,5.13908535703126E-4
PADL,DLG,703210,25513,DILLINGHAM AIRPORT,US,AK,59.05,-158.517,+0026.2,20060101,20190304,70321025513,America/Anchorage,4.404265791019735E-4
KINL,INL,727470,14918,FALLS INTERNATIONAL AIRPORT,US,MN,48.561,-93.398,+0360.6,19730101,20190304,72747014918,America/Chicago,6.057517487302925E-4
TJPS,PSE,785203,00398,MERCEDITA AIRPORT,RQ,null,18,-66.55,+0009.1,19960101,20180831,78520300398,America/Puerto_Rico,5.011645446733933E-4
KMSY,MSY,722310,12916,LOUIS ARMSTRONG NEW ORLEANS I,US,LA,29.997,-90.278,+0001.2,19451001,20190304,72231012916,America/Chicago,0.006126440774752534
NSTU,PPG,917650,61705,TAFUNA/PAGO PAGO INTERNATIONA,AQ,AS,-14.331,-170.714,+0003.7,19450801,20190303,91765061705,Pacific/Pago_Pago,4.2843050190843917E-4
KGEG,GEG,727850,24157,SPOKANE INTERNATIONAL AIRPORT,US,WA,47.622,-117.528,+0717.2,19410811,20190304,72785024157,America/Los_Angeles,0.0018568771858964643
KDRT,DRT,722610,22010,DEL RIO INTERNATIONAL AIRPORT,US,TX,29.378,-100.927,+0304.5,19510501,20190304,72261022010,America/Chicago,4.159633739412966E-4
KSNA,SNA,722977,93184,J. WAYNE APT-ORANGE CO APT,US,CA,33.68,-117.866,+0016.5,19400617,20190304,72297793184,America/Los_Angeles,0.005538200598155632
KBUR,BUR,722880,23152,BURBANK-GLENDALE-PASA ARPT,US,CA,34.201,-118.358,+0236.2,19430601,20190304,72288023152,America/Los_Angeles,0.003157212681933998


#### Broadcast the PageRank table, and use a UDF to add this feature to all airports.

In [0]:
iata_to_pr = sc.broadcast(airport_pr)

#### Note, there can be airports that we have note seen in training that show up in validation or test.

For these airports we asign it an even probability, multiplied by a teleportation factor. By doing though the sum of scores will be greater than **1.0** we are still able to represent new airports that must have been built recently or started flying minimally given they did not show up in any of the training data.

$$ pageRankNewAirport = \frac{1}{n} * telFactor$$

In [0]:
def map_iata_to_pr(iata):
  if iata in iata_to_pr.value:
    return iata_to_pr.value[iata]
  #In case the station doesn't exist we use (1/N)
  return (1/365)*.15

#convert to a UDF Function by passing in the function and return type of function
udf_map_to_pr = f.udf(map_iata_to_pr, DoubleType())
airport_meta = airport_meta.withColumn("pagerank", udf_map_to_pr("IATA"))
airport_meta.show()

+----+----+------+-----+--------------------+-------+-----+-------+--------+-------+--------+--------+-----------+-------------------+--------------------+
ICAO|IATA| usaf| wban| name|country|state| lat| lon| elev| begin| end| STATION| station_tz| pagerank|
+----+----+------+-----+--------------------+-------+-----+-------+--------+-------+--------+--------+-----------+-------------------+--------------------+
KBGM| BGM|725150|04725|GREATER BINGHAMTO...| US| NY| 42.207| -75.98|+0486.2|19730101|20190305|72515004725| America/New_York| 5.13908535703126E-4|
PADL| DLG|703210|25513| DILLINGHAM AIRPORT| US| AK| 59.05|-158.517|+0026.2|20060101|20190304|70321025513| America/Anchorage|4.404265791019735E-4|
KINL| INL|727470|14918|FALLS INTERNATION...| US| MN| 48.561| -93.398|+0360.6|19730101|20190304|72747014918| America/Chicago|6.057517487302925E-4|
TJPS| PSE|785203|00398| MERCEDITA AIRPORT| RQ| null| 18| -66.55|+0009.1|19960101|20180831|78520300398|America/Puerto_Rico|5.011645446733933E-4|
KMSY| MSY|722310|12916|LOUIS ARMSTRONG N...| US| LA| 29.997| -90.278|+0001.2|19451001|20190304|72231012916| America/Chicago|0.006126440774752534|
NSTU| PPG|917650|61705|TAFUNA/PAGO PAGO ...| AQ| AS|-14.331|-170.714|+0003.7|19450801|20190303|91765061705| Pacific/Pago_Pago|4.284305019084391...|
KGEG| GEG|727850|24157|SPOKANE INTERNATI...| US| WA| 47.622|-117.528|+0717.2|19410811|20190304|72785024157|America/Los_Angeles|0.001856877185896...|
KDRT| DRT|722610|22010|DEL RIO INTERNATI...| US| TX| 29.378|-100.927|+0304.5|19510501|20190304|72261022010| America/Chicago|4.159633739412966E-4|
KSNA| SNA|722977|93184|J. WAYNE APT-ORAN...| US| CA| 33.68|-117.866|+0016.5|19400617|20190304|72297793184|America/Los_Angeles|0.005538200598155632|
KBUR| BUR|722880|23152|BURBANK-GLENDALE-...| US| CA| 34.201|-118.358|+0236.2|19430601|20190304|72288023152|America/Los_Angeles|0.003157212681933998|
KGTF| GTF|727750|24143|GREAT FALLS INTER...| US| MT| 47.473|-111.382|+1116.8|19730101|20190304|72775024143| America/Denver|7.080113566710266E-4|
KGRB| GRB|726450|14898|AUSTIN STRAUBEL I...| US| WI| 44.479| -88.137|+0209.4|19500101|20190304|72645014898| America/Chicago|0.001028560962421...|
KIFP| IFP|723788|53135|LAUGHLIN/BULLHEAD...| US| AZ| 35.157|-114.559|+0211.8|20060101|20190304|72378853135| America/Phoenix|4.225776131210197...|
KIDA| IDA|725785|24145|IDAHO FALLS REGIO...| US| ID| 43.519|-112.064|+1442.6|19730101|20190304|72578524145| America/Boise|7.884782396195185E-4|
KGRR| GRR|726350|94860|GERALD R FORD INT...| US| MI| 42.883| -85.524|+0244.8|19730101|20190305|72635094860| America/Detroit|0.002155315884540878|
KLWB| LWB|724127|53801|GREENBRIER VALLEY...| US| WV| 37.867| -80.4|+0701.7|20060101|20190305|72412753801| America/New_York|5.599037376323678E-4|
KJLN| JLN|723495|13987|JOPLIN REGIONAL A...| US| MO| 37.152| -94.495|+0296.3|19730101|20190304|72349513987| America/Chicago|4.873310685372505...|
KPVU| PVU|725724|24174|PROVO MUNICIPAL A...| US| UT| 40.219|-111.723|+1370.7|20060101|20190304|72572424174| America/Denver|5.298675337319267E-4|
KEUG| EUG|726930|24221|MAHLON SWEET FIEL...| US| OR| 44.128|-123.221|+0107.6|19730101|20190304|72693024221|America/Los_Angeles|9.278167731853836E-4|
PAPG| PSG|703860|25329|JAMES A JOHNSON A...| US| AK| 56.806|-132.937|+0032.6|19730101|20190304|70386025329| America/Sitka|0.001061584174281...|
+----+----+------+-----+--------------------+-------+-----+-------+--------+-------+--------+--------+-----------+-------------------+--------------------+
only showing top 20 rows

In [0]:
airport_meta.write.mode('overwrite').parquet('dbfs:/mnt/mids-w261/team20SSDK/cleaned_data/station/airport_meta')

### Pagerank Spread

In [0]:
airport_meta_pd = spark.read.option("header", "true").parquet(f"dbfs:/mnt/mids-w261/team20SSDK/cleaned_data/station/airport_meta/part-00*.parquet").toPandas()
airport_meta_pd.head()

Out[3]:

,ICAO,IATA,usaf,wban,name,country,state,lat,lon,elev,begin,end,STATION,station_tz,pagerank
0,KBGM,BGM,725150,04725,GREATER BINGHAMTON/E A LINK F,US,NY,42.207,-75.98,+0486.2,19730101,20190305,72515004725,America/New_York,0.000514
1,PADL,DLG,703210,25513,DILLINGHAM AIRPORT,US,AK,59.05,-158.517,+0026.2,20060101,20190304,70321025513,America/Anchorage,0.000440
2,KINL,INL,727470,14918,FALLS INTERNATIONAL AIRPORT,US,MN,48.561,-93.398,+0360.6,19730101,20190304,72747014918,America/Chicago,0.000606
3,TJPS,PSE,785203,00398,MERCEDITA AIRPORT,RQ,None,18,-66.55,+0009.1,19960101,20180831,78520300398,America/Puerto_Rico,0.000501
4,KMSY,MSY,722310,12916,LOUIS ARMSTRONG NEW ORLEANS I,US,LA,29.997,-90.278,+0001.2,19451001,20190304,72231012916,America/Chicago,0.006126


In [0]:
fig = px.bar(airport_meta_pd.sort_values(by=["pagerank"], ascending=False), x='IATA', y='pagerank',
             hover_data=['IATA', 'pagerank', 'state','station_tz'], color='pagerank',
             title="Station Pagerank Scores", height=400)
fig.show()